# Set Approvals for Polymarket Safe

This notebook sets token approvals for a Polymarket Safe wallet using the Polymarket Relayer infrastructure.

## What it does:
1. Sets USDC approval for CTF Exchange, Neg Risk CTF Exchange, and Neg Risk Adapter
2. Sets CTF (Conditional Token Framework) approval for all three exchanges
3. Executes all approvals in a single gasless transaction (Polymarket pays the gas)

## Required Environment Variables:
- `RELAYER_PRODUCTION_URL`: Polymarket relayer URL
- `POLYGON_CHAIN_ID`: Chain ID (137 for Polygon mainnet)
- `METAMASK_EOA_PRIVATE_KEY`: Your private key
- `BUILDER_API_KEY`: Polymarket builder API key
- `BUILDER_SECRET`: Polymarket builder secret
- `BUILDER_PASS_PHRASE`: Polymarket builder passphrase
- `POLYMARKET_SAFE`: Your Polymarket Safe address

In [ ]:
import os
import dotenv

dotenv.load_dotenv()
MAIN_SAFE = "0x4c190e3C582e877942007Db3009feACc1Da6C481"
METAMASK_EOA = os.getenv("WALLET_ADDRESS")
POLYMARKET_SAFE = "0x2617746e3Da3cC26EeD139d873f0A05137023e40"

ACC_2_ADDRESS = "0xfd4fac895d20912b5d3abdd8695d891baa5e90d5"
ACC_2_PRIVATE_KEY = os.getenv("ACC_2_PRIVATE_KEY")
USDCE_ADDRESS = "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"

POLYGON_RPC = os.getenv("POLYGON_RPC")  # Polygon rpc url

TRADER_EOA_PRIVATE_KEY = os.getenv("TRADER_EOA_PRIVATE_KEY")
TRADER_EOA_SAFE_MANUAL = os.getenv("TRADER_EOA_SAFE_MANUAL")
TRADER_EOA_ADDRESS = os.getenv("TRADER_EOA_ADDRESS")


PRIVATE_KEY = os.getenv("PRIVATE_KEY")

In [ ]:
import os
import dotenv

dotenv.load_dotenv()
relayer_url = os.getenv("RELAYER_PRODUCTION_URL")
print(relayer_url)
chain_id = 137  # Polygon mainnet
private_key = PRIVATE_KEY
builder_api_key = os.getenv("BUILDER_API_KEY")
builder_secret = os.getenv("BUILDER_SECRET")
builder_passphrase = os.getenv("BUILDER_PASS_PHRASE")

polymarket_safe = MAIN_SAFE
print(polymarket_safe)

In [ ]:
# Verify Safe address and check if it's deployed
from py_builder_relayer_client.builder.derive import derive
from py_builder_relayer_client.config import get_contract_config

config = get_contract_config(chain_id)
safe_factory = config.safe_factory

# If POLYMARKET_SAFE is set in env, use it. Otherwise derive from EOA
if polymarket_safe:
    expected_safe = polymarket_safe
    print(f"Using provided Safe address: {expected_safe}")
else:
    # Derive safe address from EOA
    from web3 import Web3
    eoa_address = Web3().eth.account.from_key(private_key).address
    expected_safe = derive(eoa_address, safe_factory)
    print(f"Derived Safe address from EOA: {expected_safe}")

print(f"\nChecking if Safe is deployed...")
# Note: This will be checked by the RelayClient when executing transactions

In [ ]:
from eth_utils import to_checksum_address, keccak
from eth_abi import encode
from py_builder_relayer_client.client import RelayClient
from py_builder_relayer_client.models import OperationType, SafeTransaction
from py_builder_signing_sdk.config import BuilderConfig, BuilderApiKeyCreds

# Setup builder credentials
builder_config = BuilderConfig(
    local_builder_creds=BuilderApiKeyCreds(
        key=builder_api_key,
        secret=builder_secret,
        passphrase=builder_passphrase,
    )
)

# Initialize RelayClient
client = RelayClient(relayer_url, chain_id, private_key, builder_config)

# Contract addresses (Polygon mainnet)
usdc_address = to_checksum_address("0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174")  # USDC
ctf_address = to_checksum_address("0x4D97DCd97eC945f40cF65F87097ACe5EA0476045")  # CTF
ctf_exchange = to_checksum_address("0x4bFb41d5B3570DeFd03C39a9A4D8dE6Bd8B8982E")  # CTF Exchange
neg_risk_ctf_exchange = to_checksum_address("0xC5d563A36AE78145C45a50134d48A1215220f80a")  # Neg Risk CTF Exchange
neg_risk_adapter = to_checksum_address("0xd91E80cF2E7be2e162c6513ceD06f1dD0dA35296")  # Neg Risk Adapter

# Helper function to encode approve function call
def encode_approve(spender: str, amount: int) -> str:
    """Encode ERC20 approve function call"""
    selector = keccak(text="approve(address,uint256)")[:4]
    encoded_args = encode(["address", "uint256"], [spender, amount])
    return "0x" + (selector + encoded_args).hex()

# Helper function to encode setApprovalForAll function call
def encode_set_approval_for_all(operator: str, approved: bool) -> str:
    """Encode ERC1155 setApprovalForAll function call"""
    selector = keccak(text="setApprovalForAll(address,bool)")[:4]
    encoded_args = encode(["address", "bool"], [operator, approved])
    return "0x" + (selector + encoded_args).hex()

# Max uint256 value for unlimited approval
MAX_INT = 115792089237316195423570985008687907853269984665640564039457584007913129639935

# Create approval transactions for CTF Exchange
print("Creating approval transactions for CTF Exchange...")
usdc_approve_ctf = SafeTransaction(
    to=usdc_address,
    operation=OperationType.Call,
    data=encode_approve(ctf_exchange, MAX_INT),
    value="0"
)

ctf_approve_ctf_exchange = SafeTransaction(
    to=ctf_address,
    operation=OperationType.Call,
    data=encode_set_approval_for_all(ctf_exchange, True),
    value="0"
)

# Create approval transactions for Neg Risk CTF Exchange
print("Creating approval transactions for Neg Risk CTF Exchange...")
usdc_approve_neg_risk = SafeTransaction(
    to=usdc_address,
    operation=OperationType.Call,
    data=encode_approve(neg_risk_ctf_exchange, MAX_INT),
    value="0"
)

ctf_approve_neg_risk = SafeTransaction(
    to=ctf_address,
    operation=OperationType.Call,
    data=encode_set_approval_for_all(neg_risk_ctf_exchange, True),
    value="0"
)

# Create approval transactions for Neg Risk Adapter
print("Creating approval transactions for Neg Risk Adapter...")
usdc_approve_adapter = SafeTransaction(
    to=usdc_address,
    operation=OperationType.Call,
    data=encode_approve(neg_risk_adapter, MAX_INT),
    value="0"
)

ctf_approve_adapter = SafeTransaction(
    to=ctf_address,
    operation=OperationType.Call,
    data=encode_set_approval_for_all(neg_risk_adapter, True),
    value="0"
)

# Execute all approval transactions together
print("\nExecuting all approval transactions...")
transactions = [
    usdc_approve_ctf,
    ctf_approve_ctf_exchange,
    usdc_approve_neg_risk,
    ctf_approve_neg_risk,
    usdc_approve_adapter,
    ctf_approve_adapter
]

response = client.execute(transactions, "Set all Polymarket approvals for Safe")
print(f"Transaction submitted!")
print(f"Transaction ID: {response.transaction_id}")
print(f"Transaction Hash: {response.transaction_hash}")

# Wait for transaction to be confirmed
print("\nWaiting for transaction confirmation...")
result = response.wait()

if result:
    print(f"\n✅ All approvals set successfully!")
    print(f"Transaction Hash: {result.get('transactionHash')}")
    print(f"Transaction State: {result.get('state')}")
else:
    print("\n❌ Transaction failed or timed out")

In [ ]:
# Check if allowances were set successfully
from web3 import Web3
from eth_abi import decode

# Connect to Polygon RPC
rpc_url = "https://polygon-rpc.com"  # You can use any Polygon RPC
w3 = Web3(Web3.HTTPProvider(rpc_url))

# Contract addresses
usdc_address = "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"
ctf_address = "0x4D97DCd97eC945f40cF65F87097ACe5EA0476045"
ctf_exchange = "0x4bFb41d5B3570DeFd03C39a9A4D8dE6Bd8B8982E"
neg_risk_ctf_exchange = "0xC5d563A36AE78145C45a50134d48A1215220f80a"
neg_risk_adapter = "0xd91E80cF2E7be2e162c6513ceD06f1dD0dA35296"

# Your Safe address
safe_address = polymarket_safe if polymarket_safe else expected_safe

print(f"Checking allowances for Safe: {safe_address}\n")

# Check transaction receipt first
tx_hash = "0x52d201123a311f64b68041a0fcdbb606345095d40e62eaf01635b76bfab297f5"
try:
    receipt = w3.eth.get_transaction_receipt(tx_hash)
    print(f"✅ Transaction was mined in block {receipt['blockNumber']}")
    print(f"   Status: {'Success' if receipt['status'] == 1 else 'Failed'}")
    print(f"   Gas used: {receipt['gasUsed']}\n")
except Exception as e:
    print(f"⚠️  Could not fetch transaction receipt: {e}\n")

# ERC20 allowance function signature
allowance_sig = w3.keccak(text="allowance(address,address)")[:4].hex()

# ERC1155 isApprovedForAll function signature  
is_approved_sig = w3.keccak(text="isApprovedForAll(address,address)")[:4].hex()

def check_erc20_allowance(token_address, owner, spender):
    """Check ERC20 allowance"""
    data = allowance_sig + encode(["address", "address"], [owner, spender]).hex()
    result = w3.eth.call({
        "to": w3.to_checksum_address(token_address),
        "data": data
    })
    allowance = int.from_bytes(result, byteorder='big')
    return allowance

def check_erc1155_approval(token_address, owner, operator):
    """Check ERC1155 approval"""
    data = is_approved_sig + encode(["address", "address"], [owner, operator]).hex()
    result = w3.eth.call({
        "to": w3.to_checksum_address(token_address),
        "data": data
    })
    is_approved = int.from_bytes(result, byteorder='big') == 1
    return is_approved

print("USDC Allowances:")
print(f"  CTF Exchange: {check_erc20_allowance(usdc_address, safe_address, ctf_exchange):,}")
print(f"  Neg Risk CTF Exchange: {check_erc20_allowance(usdc_address, safe_address, neg_risk_ctf_exchange):,}")
print(f"  Neg Risk Adapter: {check_erc20_allowance(usdc_address, safe_address, neg_risk_adapter):,}")

print("\nCTF Approvals:")
print(f"  CTF Exchange: {check_erc1155_approval(ctf_address, safe_address, ctf_exchange)}")
print(f"  Neg Risk CTF Exchange: {check_erc1155_approval(ctf_address, safe_address, neg_risk_ctf_exchange)}")
print(f"  Neg Risk Adapter: {check_erc1155_approval(ctf_address, safe_address, neg_risk_adapter)}")